In [2]:
import joblib
import tarfile
import h5py 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyechonest
import librosa
import librosa.display
import io
from urllib.request import urlopen
import pydub
import random
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = ''
secret = ''
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
def song_extraction(spotipy_search = False, q='', id_list = [], url_list = [], limit = 50, just_raw_array=False, just_spotify = False, sample_rate=22050,size=25800,with_nan_arrays = False):
    if spotipy_search: 
        ids = []
        prevs = []
        for i in (sp.search(q=q,type='track',limit=limit)['tracks']['items']):
            prevs.append(i['preview_url'])
            ids.append(i['uri'][-22:])
        true_inds = np.where(np.array(prevs) != None)
        ids = [ids[i] for i in true_inds[0]]
        prevs = [i for i in prevs if i != None]
    else: 
        ids = id_list.copy()
        prevs = url_list.copy()
    

    wav_array = []
    for mp3_url in prevs:
        wav = io.BytesIO()
        try:
            with urlopen(mp3_url) as r:
                r.seek = lambda *args: None  # allow pydub to call seek(0)
                pydub.AudioSegment.from_file(r).export(wav, "wav")

            wav.seek(0)
            y, sr = librosa.load(wav)
            y = y[:660984]
            wav_array.append(y)
        except:
            pass
    if just_raw_array:
        #mfccs = []
        #mels = []
        #for i in wav_array:
            #temp_mfcc = librosa.feature.mfcc(y=i,sr=sample_rate)
            #temp_mfcc = temp_mfcc.reshape((temp_mfcc.shape[0]*temp_mfcc.shape[1],1))
            #temp_mfcc = temp_mfcc[0:size,:]

            #temp_mel = librosa.power_to_db(librosa.feature.melspectrogram(y=i,sr=sample_rate),ref=np.max)
            ###temp_mel = temp_mel.reshape((temp_mel.shape[0]*temp_mel.shape[1],1))
            ###temp_mel = temp_mel[0:size,:]
            #mfccs.append(temp_mfcc)
            #mels.append(temp_mel)
        return pd.DataFrame(zip(ids,wav_array),columns=['id','raw_array'])
        
        
    if just_spotify:
        features_dict = []
        for i in ids: 
            temp = sp.track(i)
            an = sp.audio_features(i)
            features_dict.append({'id':i,'explicit':temp['explicit'],'popularity':temp['popularity'],'danceability':an[0]['danceability'],'energy':an[0]['energy'],'key':an[0]['key'], 'loudness':an[0]['loudness'], 'mode':an[0]['mode'], 'speechiness':an[0]['speechiness'], 'acousticness':an[0]['acousticness'], 'instrumentalness':an[0]['instrumentalness'], 'liveness':an[0]['liveness'], 'valence':an[0]['valence'], 'tempo':an[0]['tempo']})
        df = pd.DataFrame(features_dict)
        ar = pd.DataFrame(zip(ids,wav_array),columns=['id','raw_array'])
        return df.merge(ar,on='id')
        
    else:
        mfccs = []
        mels = []
        stft = []
        cqt = []
        cens = []
        rms = []
        spectral_centroid = []
        spectral_bandwidth = []
        spectral_contrast = []
        spectral_flatness = [] 
        spectral_rolloff = [] 
        poly_features = [] 
        tonnetz = [] 
        zcr = [] 
        tempo = [] 
        fourier = [] 
        delta = [] 
        stack = [] 
        onset_det = []
        strength = []
        strength_m = []
        beat = [] 
        plp = [] 
        tempo = [] 
        hpss = [] 
        harmonic = [] 
        percussive = [] 


        for i in wav_array:
            temp_mfcc = librosa.feature.mfcc(y=i,sr=sample_rate)
            temp_mfcc = temp_mfcc.reshape((temp_mfcc.shape[0]*temp_mfcc.shape[1],1))
            temp_mfcc = temp_mfcc[0:size,:]

            temp_mel = librosa.power_to_db(librosa.feature.melspectrogram(y=i,sr=sample_rate),ref=np.max)
            #temp_mel = temp_mel.reshape((temp_mel.shape[0]*temp_mel.shape[1],1))
            #temp_mel = temp_mel[0:size,:]
            mfccs.append(temp_mfcc)
            mels.append(temp_mel)

            stft.append(librosa.feature.chroma_stft(y=i,sr=sample_rate))
            cqt.append(librosa.feature.chroma_cqt(y=i,sr=sample_rate))
            cens.append(librosa.feature.chroma_cens(y=i,sr=sample_rate))
            rms.append(librosa.feature.rms(y=i))
            spectral_centroid.append(librosa.feature.spectral_centroid(y=i,sr=sample_rate))
            spectral_bandwidth.append(librosa.feature.spectral_bandwidth(y=i,sr=sample_rate))
            spectral_contrast.append(librosa.feature.spectral_contrast(y=i,sr=sample_rate))
            spectral_flatness.append(librosa.feature.spectral_flatness(y=i))
            spectral_rolloff.append(librosa.feature.spectral_rolloff(y=i,sr=sample_rate))
            poly_features.append(librosa.feature.poly_features(y=i,sr=sample_rate))
            tonnetz.append(librosa.feature.tonnetz(y=i,sr=sample_rate))
            zcr.append(librosa.feature.zero_crossing_rate(y=i))
            tempo.append(librosa.feature.tempogram(y=i,sr=sample_rate))
            fourier.append(librosa.feature.fourier_tempogram(y=i,sr=sample_rate))
            delta.append(librosa.feature.delta(data=i))
            stack.append(librosa.feature.stack_memory(data=i))
            onset_det.append(librosa.onset.onset_detect(y=i,sr=sample_rate))
            strength.append(librosa.onset.onset_strength(y=i,sr=sample_rate))
            strength_m.append(librosa.onset.onset_strength_multi(y=i,sr=sample_rate))
            beat.append(librosa.beat.beat_track(y=i,sr=sample_rate))
            plp.append(librosa.beat.plp(y=i,sr=sample_rate))
            tempo.append(librosa.beat.tempo(y=i,sr=sample_rate))
            hpss.append(librosa.effects.hpss(y=i))
            harmonic.append(librosa.effects.harmonic(y=i))
            percussive.append(librosa.effects.percussive(y=i))




        array_df = pd.DataFrame(zip(wav_array,mfccs,mels,stft,cqt,cens,rms,spectral_centroid,
                                    spectral_bandwidth,spectral_contrast,spectral_flatness,spectral_rolloff,
                                    poly_features, tonnetz, zcr, tempo,fourier,delta,stack,
                                    onset_det, strength, strength_m,
                                    beat, plp, tempo, hpss, harmonic, percussive
                                    ),
                                columns=['raw_array','mfcc','mel','stft','cqt','cens','rms',
                                         'spectral_centroid','spectral_bandwidth','spectral_contrast',
                                         'spectral_flatness','spectral_rolloff',
                                         'poly_features','tonnetz','zero_crossing_rate',
                                         'tempogram','fourier_tempogram','delta','stack_memory',
                                         'onset_detect','onset_strength','onset_strength_multi',
                                         'beat_track','plp','tempo_ar','hpss','harmonic','percussive'])

        features_dict = []
        for i in ids: 
            temp = sp.track(i)
            an = sp.audio_features(i)
            features_dict.append({'id':i,'name':temp['name'],'explicit':temp['explicit'],'popularity':temp['popularity'],'danceability':an[0]['danceability'],'energy':an[0]['energy'],'key':an[0]['key'], 'loudness':an[0]['loudness'], 'mode':an[0]['mode'], 'speechiness':an[0]['speechiness'], 'acousticness':an[0]['acousticness'], 'instrumentalness':an[0]['instrumentalness'], 'liveness':an[0]['liveness'], 'valence':an[0]['valence'], 'tempo':an[0]['tempo']})
        df = pd.concat([pd.DataFrame(features_dict),array_df],axis=1)
        if with_nan_arrays:
            return df 
        else:
            return df[df['raw_array'] != np.nan]

In [6]:
#dir(sp)

In [5]:
genre = pd.read_csv('genresongs.csv')
genre = genre.iloc[:,1:]

In [6]:
pop = pd.read_csv('popular.csv')
pop = pop.iloc[:,1:]

In [6]:
for i,j in zip(range(0,len(pop)-500,500),range(500,len(pop),500)):
    spot_temp = song_extraction(id_list = pop['id'][i:j] , url_list = pop['url'][i:j], just_spotify=True, sample_rate=22050,size=25800)
    temp = spot_temp.merge(pop,on='id')
    feats = temp.drop('raw_array',axis=1)
    array = np.vstack(list(temp['raw_array']))

    feats_name = r'Pop_Array_feats/pop_feats_' + str(i) + '.csv'
    array_name = r'Pop_Array_feats/pop_array_' + str(i) + '.csv'
    
    feats.to_csv(feats_name)
    pd.DataFrame(array).to_csv(array_name)

In [10]:
i = 3500
spot_temp = song_extraction(id_list = pop['id'][i:] , url_list = pop['url'][i:], just_spotify=True, sample_rate=22050,size=25800)
temp = spot_temp.merge(pop,on='id')
feats = temp.drop('raw_array',axis=1)
array = np.vstack(list(temp['raw_array']))

feats_name = r'Pop_Array_feats/pop_feats_' + str(i) + '.csv'
array_name = r'Pop_Array_feats/pop_array_' + str(i) + '.csv'

feats.to_csv(feats_name)
pd.DataFrame(array).to_csv(array_name)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 660984 and the array at index 262 has size 516984

In [14]:
temp = temp.drop(index=[262])

In [15]:
array = np.vstack(list(temp['raw_array']))

feats_name = r'Pop_Array_feats/pop_feats_' + str(i) + '.csv'
array_name = r'Pop_Array_feats/pop_array_' + str(i) + '.csv'

feats.to_csv(feats_name)
pd.DataFrame(array).to_csv(array_name)

In [7]:
random.seed(1993)
sample_ids = []
for i in genre['genre'].unique():
    temp = genre[genre['genre']==i]
    [sample_ids.append(i) for i in random.sample(list(temp['id']),50)]

sample_input = genre[genre['id'].isin(sample_ids)]
sample_input



,id,name,artist,url,genre
7,69Rq57lrGf5syykFBNi3g4,Hope,Twista,https://listen.hs.llnwd.net/g2/prvw/7/7/4/8/1/...,Hip hop
15,4gQdwwPXSXnDkUh4jYqLoC,Hip Hop,Mos Def,NaN,Hip hop
33,6rVujfYLm9YmdyjV5xTgsS,Ill Mind of Hopsin 7,Hopsin,https://listen.hs.llnwd.net/g3/prvw/3/7/4/0/7/...,Hip hop
37,7J2T6mw5zBGhHLsK3GC5PA,The Purge,Hopsin,https://listen.hs.llnwd.net/g3/prvw/4/1/1/4/5/...,Hip hop
73,3lowcuAcg6INX4Rs32KzjJ,Hop is Back,Hopsin,https://listen.hs.llnwd.net/g1/prvw/7/0/6/0/0/...,Hip hop
...,...,...,...,...,...
4652,6IwKcFdiRQZOWeYNhUiWIv,No One,Alicia Keys,https://listen.hs.llnwd.net/g1/prvw/4/7/1/3/4/...,Pop
4690,5bcTCxgc7xVfSaMV3RuVke,"Feels (feat. Pharrell Williams, Katy Perry & B...",Calvin Harris,https://listen.hs.llnwd.net/g3/prvw/4/9/9/7/4/...,Pop
4770,4XNrMwGx1SqP01sqkGTDmo,One More Night,Maroon 5,https://listen.hs.llnwd.net/g3/prvw/2/2/2/3/7/...,Pop
4840,60gTdTwaNtGAzIxKfeGVfJ,Love's Train,Bruno Mars,https://listen.hs.llnwd.net/g3/prvw/4/5/5/0/4/...,Pop


In [8]:
sample_input.genre.value_counts()

Pop        54
EDM        52
Blues      51
Hip hop    50
Metal      50
Name: genre, dtype: int64

In [9]:

temp = song_extraction(id_list = sample_input['id'] , url_list = sample_input['url'], just_spotify=True, sample_rate=22050,size=25800)

KeyError: "['name_y'] not found in axis"

In [12]:
output = sample_input.merge(temp,on='id')
#sample_input.merge(sample,on='id')

feats = output.drop('raw_array',axis=1)
array = np.vstack(list(output['raw_array']))

feats.to_csv(r'genre_sample_50each/features.csv')
pd.DataFrame(array).to_csv(r'genre_sample_50each/raw_arrays.csv')

In [11]:
temp


,id,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,raw_array
0,69Rq57lrGf5syykFBNi3g4,True,61,0.900,0.656,8,-7.218,0,0.2220,0.4300,0.000000,0.1890,0.288,106.948,"[1.05128496e-10, -1.1958694e-10, 1.2033916e-10..."
1,4gQdwwPXSXnDkUh4jYqLoC,True,58,0.592,0.861,1,-5.177,1,0.2490,0.0240,0.000000,0.4920,0.794,90.238,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,6rVujfYLm9YmdyjV5xTgsS,True,54,0.642,0.791,2,-3.859,0,0.3220,0.0515,0.000000,0.1190,0.331,93.015,"[-1.731929e-05, 6.4626274e-06, 2.7337841e-05, ..."
3,7J2T6mw5zBGhHLsK3GC5PA,True,54,0.714,0.844,7,-8.031,1,0.3090,0.2150,0.000000,0.1300,0.543,147.786,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,3lowcuAcg6INX4Rs32KzjJ,True,50,0.578,0.864,5,-6.153,0,0.3830,0.1070,0.000000,0.4810,0.649,175.775,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,6gU9OKjOE7ghfEd55oRO57,True,76,0.400,0.366,8,-9.405,0,0.0597,0.2420,0.000000,0.1260,0.161,73.313,"[-9.3078475e-11, 1.3504386e-10, -1.8780819e-10..."
257,6t6oULCRS6hnI7rm0h5gwl,True,74,0.672,0.738,0,-7.045,1,0.0506,0.0178,0.000068,0.0927,0.392,107.938,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
258,1jVjOcs7QeQZJXScbqvofI,True,75,0.606,0.532,3,-6.076,1,0.0416,0.4400,0.000000,0.1320,0.405,81.332,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
259,3JvKfv6T31zO0ini8iNItO,True,92,0.445,0.537,4,-8.532,0,0.0400,0.6950,0.000017,0.0944,0.131,122.769,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
